# Lista de empresas

🔍 Busca inicial Obtém a lista completa de ativos disponíveis na BRAPI.

🧹 Filtragem criteriosa

Mantém apenas ativos dos tipos "stock" e "fund".

Exclui tickers fracionados (aqueles terminados em F.SA).

Considera exclusivamente ativos cujo país de origem seja Brasil, validado via yfinance.

Restringe a análise a setores específicos: Finance, Utilities, Communications e Industrial Services.

🧠 Validação e enriquecimento

Usa yfinance para confirmar o país e obter o nome completo da empresa.

Traduz os nomes dos setores para o padrão da B3.

Traduz automaticamente os subsetores de inglês para português brasileiro usando a biblioteca deep-translator.

Monta um dicionário com os principais atributos: ticker, empresa, setor, subsetor, tipo, logo.

💾 Exportação final

Organiza os dados em estrutura limpa e ordenada por setor e valor de mercado.

Salva os resultados em um arquivo .csv, com o caminho de saída definido dinamicamente via variável no arquivo .env.

In [1]:
import requests
import yfinance as yf
import pandas as pd
import os
import warnings
from dotenv import load_dotenv
from deep_translator import GoogleTranslator

warnings.simplefilter(action='ignore', category=FutureWarning)
load_dotenv()

# 🎯 Setores-alvo
SETORES_ALVO = {
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
}

# 🌐 Tradução de setores para padrão B3
TRADUCAO_SETORES = {
    "Finance": "Financeiro",
    "Utilities": "Utilidade Pública",
    "Communications": "Comunicações",
    "Industrial Services": "Serviços Industriais"
}

# 🏷️ Tradução de tipos de ativos
TRADUCAO_TIPOS = {
    "stock": "Ação",
    "fund": "Fundo"
}

# 🌍 Função para traduzir subsetores
def traduzir_subsetor(texto):
    if not texto or texto == "N/A":
        return "N/A"
    try:
        return GoogleTranslator(source='en', target='pt').translate(texto)
    except Exception:
        return texto  # fallback para o original se falhar

# 🔍 Função principal
def coletar_dados_ativos(setores):
    try:
        lista = requests.get("https://brapi.dev/api/quote/list").json().get("stocks", [])
    except Exception as e:
        print(f"❌ Erro ao buscar ativos: {e}")
        return pd.DataFrame()

    ativos = {
        f"{item['stock']}.SA": {
            "setor_brapi": TRADUCAO_SETORES.get(item["sector"], item["sector"]),
            "tipo": item["type"],
            "logo": item.get("logo")
        }
        for item in lista
        if item["sector"] in setores and item["type"] in {"stock", "fund"} and not item["stock"].endswith("F")
    }

    registros = []
    for i, (ticker, dados) in enumerate(ativos.items(), 1):
        try:
            info = yf.Ticker(ticker).info
            if info.get("country") != "Brazil":
                continue

            subsetor_en = info.get("industry", "N/A")
            subsetor_pt = traduzir_subsetor(subsetor_en)

            registros.append({
                "ticker": ticker,
                "empresa": info.get("longName", "N/A"),
                "setor_brapi": dados["setor_brapi"],
                "subsetor_yfinance": subsetor_pt,
                "pais": info.get("country", "N/A"),
                "tipo": TRADUCAO_TIPOS.get(dados["tipo"], dados["tipo"]),  # Aplicando tradução aqui
                "market_cap": info.get("marketCap"),
                "logo": dados["logo"]
            })
            print(f"✅ [{i}] {ticker} incluído")
        except Exception:
            continue

    return pd.DataFrame(registros)

# 💾 Execução e salvamento
df = coletar_dados_ativos(SETORES_ALVO)


✅ [1] BBAS3.SA incluído
✅ [2] B3SA3.SA incluído
✅ [3] BBDC4.SA incluído
✅ [4] ITUB4.SA incluído
✅ [5] MOVI3.SA incluído
✅ [6] CSAN3.SA incluído
✅ [7] ELET3.SA incluído
✅ [8] ITSA4.SA incluído
✅ [9] CPLE6.SA incluído
✅ [10] VAMO3.SA incluído
✅ [11] BPAC11.SA incluído
✅ [12] RENT3.SA incluído
✅ [13] CMIG4.SA incluído
✅ [14] ENEV3.SA incluído
✅ [15] EQTL3.SA incluído
✅ [16] TEND3.SA incluído
✅ [17] CPLE3.SA incluído
✅ [18] BBSE3.SA incluído
✅ [19] AURE3.SA incluído
✅ [20] ELET6.SA incluído
✅ [21] ALOS3.SA incluído
✅ [22] BBDC3.SA incluído
✅ [23] CYRE3.SA incluído
✅ [24] CURY3.SA incluído
✅ [25] VIVT3.SA incluído
✅ [26] TIMS3.SA incluído
✅ [27] MULT3.SA incluído
✅ [28] CPFE3.SA incluído
✅ [29] BRAP4.SA incluído
✅ [30] SBSP3.SA incluído
✅ [31] CSMG3.SA incluído
✅ [32] CXSE3.SA incluído
✅ [33] SRNA3.SA incluído
✅ [34] ENGI11.SA incluído
✅ [35] SAPR11.SA incluído
✅ [36] AZTE3.SA incluído
✅ [37] PSSA3.SA incluído
✅ [38] EZTC3.SA incluído
✅ [39] JHSF3.SA incluído
✅ [40] SAPR4.SA incluído
✅ [41]

# Dividendos

In [ ]:
import os
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Carrega variáveis do .env
load_dotenv()

# Caminho do arquivo CSV com os tickers
csv_path = os.getenv("OUTPUT_PATH")

# Lê o CSV e extrai os tickers
df = pd.read_csv(csv_path)
tickers = df['Ticker'].dropna().unique().tolist()

# Define o período de 7 anos
end_date = datetime.today()
start_date = end_date - timedelta(days=7*365)

todos_dividendos = []

for ticker in tickers:
    try:
        acao = yf.Ticker(ticker)
        dividendos = acao.dividends

        # Remove timezone do índice
        dividendos.index = dividendos.index.tz_localize(None)

        # Filtra dividendos dos últimos 7 anos
        dividendos = dividendos[dividendos.index >= start_date]

        if not dividendos.empty:
            df_div = dividendos.reset_index()
            df_div.columns = ['Data', 'Valor']
            df_div['Ticker'] = ticker.replace('.SA', '')
            todos_dividendos.append(df_div)

            print(f"✅ Dividendos coletados para {ticker}")
        else:
            print(f"⚠️ Sem dividendos recentes para {ticker}")
    except Exception as e:
        print(f"❌ Erro ao processar {ticker}: {e}")

# Junta tudo em um único DataFrame
df_final = pd.concat(todos_dividendos, ignore_index=True)

# Salva em um único CSV
df_final.to_csv("../data/todos_dividendos.csv", index=False)

print("🏁 Finalizado! Dividendos salvos em todos_dividendos.csv")


### Dividendo por ano

In [ ]:
import pandas as pd

# Ler o arquivo CSV
df = pd.read_csv('../data/todos_dividendos.csv')

# Converter a coluna 'Data' para datetime e extrair o ano
df['Data'] = pd.to_datetime(df['Data'])
df['ano'] = df['Data'].dt.year

# Renomear colunas para minúsculas
df = df.rename(columns={'Ticker': 'ticket', 'Valor': 'dividendo', 'Data': 'data'})

# Somar os dividendos por ano e ticket
soma_por_ano_ticket = df.groupby(['ano', 'ticket'])['dividendo'].sum().reset_index()

# Salvar o resultado em um CSV
soma_por_ano_ticket.to_csv('../data/dividendos_ano.csv', index=False)
print("Arquivo 'dividendos_ano.csv' gerado com sucesso!")

In [ ]:
import pandas as pd 

div_ano = pd.read_csv('../data/dividendos_ano.csv')
print(div_ano.head())

### Dividendos por ano resumido (DY5 e DY12M)

In [ ]:
import pandas as pd

# Carregar os dividendos por ano e ticket
df = pd.read_csv('../data/dividendos_ano.csv')

# Descobrir o último ano disponível
ultimo_ano = df['ano'].max()

# Dividendos dos últimos 5 anos
div_5anos = df[df['ano'] >= ultimo_ano - 4]
soma_5anos = div_5anos.groupby('ticket')['dividendo'].sum().reset_index()
soma_5anos = soma_5anos.rename(columns={'dividendo': 'valor_5anos'})

# Dividendos dos últimos 12 meses
div_12m = df[df['ano'] == ultimo_ano]
soma_12m = div_12m[['ticket', 'dividendo']].rename(columns={'dividendo': 'valor_12m'})

# Juntar os dois
resumo = pd.merge(soma_5anos, soma_12m, on='ticket', how='outer').fillna(0)

# Reorganizar colunas
resumo = resumo[['ticket', 'valor_5anos', 'valor_12m']]

# Salvar resultado
resumo.to_csv('../data/dividendos_ano_resumo.csv', index=False)
print("Arquivo 'dividendos_ano_resumo.csv' gerado com sucesso!")
print(resumo.head())
print(resumo.shape)


## Extraindo valores das ações

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date
import warnings
import os
from dotenv import load_dotenv

# Carregar as variáveis do arquivo .env
load_dotenv()

# Ignora avisos futuros para manter o output limpo
warnings.simplefilter(action='ignore', category=FutureWarning)

def ler_tickers_do_csv(caminho_do_arquivo: str, coluna_ticker: str = 'ticker') -> list:
    """Lê um arquivo CSV e extrai uma lista de tickers únicos."""
    try:
        df = pd.read_csv(caminho_do_arquivo)
        if coluna_ticker not in df.columns:
            print(f"Erro: A coluna '{coluna_ticker}' não foi encontrada no arquivo.")
            print(f"Colunas disponíveis: {list(df.columns)}")
            return []
        tickers = df[coluna_ticker].dropna().unique().tolist()
        return tickers
    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em '{caminho_do_arquivo}'")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo: {e}")
        return []

def gerar_tabela_comparativa_precos(lista_tickers: list, anos_anteriores: int = 7) -> tuple[pd.DataFrame, pd.DataFrame] | tuple[None, None]:
    """Busca o preço de fechamento ajustado para uma lista de tickers."""
    try:
        # Verifica se o ticker já tem .SA, se não tiver, adiciona
        tickers_sa = []
        for t in lista_tickers:
            ticker_upper = t.upper()
            if ticker_upper.endswith('.SA'):
                tickers_sa.append(ticker_upper)
            else:
                tickers_sa.append(f"{ticker_upper}.SA")
        
        hoje = date.today()
        ano_inicio = hoje.year - anos_anteriores
        
        print(f"Baixando dados históricos de {len(tickers_sa)} ativos...")
        hist = yf.download(tickers_sa, start=f"{ano_inicio}-01-01", end=hoje, auto_adjust=True, progress=False)
        
        if hist.empty:
            print("Nenhum dado histórico foi encontrado.")
            return None, None
            
        df_closes = hist['Close']
        
        # Lista para tabela completa (ticker, ano, fechamento)
        lista_completa = []
        # Lista para tabela resumida (ticker, fechamento_atual)
        lista_resumida = []
        
        for i, ticker in enumerate(lista_tickers):
            ticker_sa = tickers_sa[i]  # Usa o ticker já processado
            
            if ticker_sa not in df_closes.columns or df_closes[ticker_sa].dropna().empty:
                print(f"Aviso: Não foram encontrados dados para o ticker {ticker}. Pulando.")
                continue

            # Para tabela resumida - apenas fechamento atual
            fechamento_atual = df_closes[ticker_sa].iloc[-1].item()
            lista_resumida.append({
                'ticker': ticker,
                'fechamento_atual': fechamento_atual
            })
            
            # Para tabela completa - todos os anos
            # Adicionar fechamento atual
            lista_completa.append({
                'ticker': ticker,
                'ano': hoje.year,
                'fechamento': fechamento_atual
            })
            
            # Adicionar anos anteriores
            for j in range(anos_anteriores):
                ano_alvo = hoje.year - (j + 1)
                
                try:
                    dados_ano = df_closes.loc[str(ano_alvo)]
                    
                    if not dados_ano.empty and not dados_ano[ticker_sa].dropna().empty:
                        fechamento_ano = dados_ano[ticker_sa].iloc[-1].item()
                        lista_completa.append({
                            'ticker': ticker,
                            'ano': ano_alvo,
                            'fechamento': fechamento_ano
                        })
                except KeyError:
                    # Ano não encontrado nos dados - adiciona com None
                    lista_completa.append({
                        'ticker': ticker,
                        'ano': ano_alvo,
                        'fechamento': None
                    })

        if not lista_completa:
            print("Nenhum resultado foi processado com sucesso.")
            return None, None

        # Criar DataFrames
        df_completo = pd.DataFrame(lista_completa)
        df_resumido = pd.DataFrame(lista_resumida).set_index('ticker')
        
        return df_completo, df_resumido

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None, None

# --- BLOCO DE EXECUÇÃO PRINCIPAL ---

# Pega o caminho do arquivo a partir da variável de ambiente
csv_path = os.getenv("FILTERED_COMPANIES_PATH")

if csv_path:
    print(f"Usando arquivo: {csv_path}")
    
    ativos_alvo = ler_tickers_do_csv(csv_path)

    if ativos_alvo:
        print(f"Processando {len(ativos_alvo)} ativos encontrados no arquivo: {', '.join(ativos_alvo[:5])}...")
        
        # Define o número de anos aqui para usar depois
        anos_para_analise = 7
        tabela_completa, tabela_resumida = gerar_tabela_comparativa_precos(ativos_alvo, anos_anteriores=anos_para_analise)

        if tabela_completa is not None and tabela_resumida is not None:
            print("\n--- Exibição Resumida (5 primeiras linhas) ---")
            
            # Exibir apenas 5 linhas da tabela resumida com 2 casas decimais
            tabela_exibicao = tabela_resumida.head(5).copy()
            tabela_exibicao['fechamento_atual'] = tabela_exibicao['fechamento_atual'].apply(
                lambda x: f"R$ {x:.2f}" if pd.notna(x) else "N/D"
            )
            print(tabela_exibicao.to_string())
            
            # Definir pasta de saída
            output_folder = r"E:\Github\Unicamp\Bussola-de-Valor\data"
            os.makedirs(output_folder, exist_ok=True)
            
            # Salvar tabela completa (ticker, ano, fechamento)
            output_path_completo = os.path.join(output_folder, "precos_acoes_completo.csv")
            tabela_completa_salvar = tabela_completa.copy()
            tabela_completa_salvar['fechamento'] = tabela_completa_salvar['fechamento'].round(2)
            tabela_completa_salvar.to_csv(output_path_completo, index=False)
            print(f"\n✅ Tabela completa salva em: {output_path_completo}")
            
            # Salvar tabela resumida (ticker, fechamento_atual)
            output_path_resumido = os.path.join(output_folder, "precos_acoes.csv")
            tabela_resumida_salvar = tabela_resumida.round(2)
            tabela_resumida_salvar.to_csv(output_path_resumido)
            print(f"✅ Tabela resumida salva em: {output_path_resumido}")
            
            # Estatísticas básicas
            print(f"\n📊 Estatísticas:")
            print(f"   • Total de ativos processados: {len(tabela_resumida)}")
            print(f"   • Período analisado: {anos_para_analise} anos")
            print(f"   • Total de registros na tabela completa: {len(tabela_completa)}")
            print(f"   • Data de execução: {date.today().strftime('%d/%m/%Y')}")
            
        else:
            print("Não foi possível gerar as tabelas de preços.")
    else:
        print("Nenhum ativo para processar. Verifique o conteúdo do arquivo CSV.")
else:
    print("ERRO: A variável de ambiente 'FILTERED_COMPANIES_PATH' não está definida no seu arquivo .env ou no sistema.")
    print("Certifique-se de que existe um arquivo .env com a linha:")
    print("FILTERED_COMPANIES_PATH=caminho/para/seu/arquivo.csv")


In [ ]:
## Rever o codigo abaixo

## Script de Transformação

In [ ]:
# transform.py (Versão com Cálculo de DY Padronizado)
import yfinance as yf
import pandas as pd
import os
from datetime import datetime

def classify_stock_profile(price: float, market_cap: float) -> str:
    """
    Classifica a ação com base no Market Cap e Preço.
    """
    if price is not None and price < 1.0:
        return "Penny Stock"
    if market_cap is None or market_cap == 0:
        return "N/A"
    if market_cap > 50_000_000_000:
        return "Blue Chip"
    if market_cap > 10_000_000_000:
        return "Mid Cap"
    if market_cap > 2_000_000_000:
        return "Small Cap"
    return "Micro Cap"

def get_market_sentiment_and_details(ticker_obj: yf.Ticker) -> dict:
    """
    Usa 'recommendations_summary' para maior robustez.
    """
    sentiment_data = {
        'Sentimento Gauge': 50.0, 'Strong Buy': 0, 'Buy': 0,
        'Hold': 0, 'Sell': 0, 'Strong Sell': 0
    }
    try:
        summary = ticker_obj.recommendations_summary
        if summary is None or summary.empty:
            return sentiment_data
        rec_counts = summary.iloc[-1]
        strong_buy = int(rec_counts.get('strongBuy', 0))
        buy = int(rec_counts.get('buy', 0))
        hold = int(rec_counts.get('hold', 0))
        sell = int(rec_counts.get('sell', 0))
        strong_sell = int(rec_counts.get('strongSell', 0))
        sentiment_data.update({
            'Strong Buy': strong_buy, 'Buy': buy, 'Hold': hold,
            'Sell': sell, 'Strong Sell': strong_sell
        })
        weighted_score = (strong_buy * 2) + (buy * 1) + (hold * 0) + (sell * -1) + (strong_sell * -2)
        total_recommendations = strong_buy + buy + hold + sell + strong_sell
        if total_recommendations == 0:
            return sentiment_data
        avg_score = weighted_score / total_recommendations
        normalized_sentiment = ((avg_score + 2) / 4) * 100
        sentiment_data['Sentimento Gauge'] = normalized_sentiment
        return sentiment_data
    except Exception as e:
        print(f" (Aviso: Falha ao buscar sentimento - {e}) ", end="")
        return sentiment_data

def fetch_stock_data(ticker: str, info_original: pd.Series) -> dict:
    """
    Busca e processa os dados de um único ticker, com lógica de fallback para Market Cap e DY 5 anos.
    """
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="5y")

    market_cap = info.get('marketCap')
    current_price = info.get("currentPrice", 0.0)
    if not market_cap or market_cap == 0:
        shares_outstanding = info.get('sharesOutstanding', 0)
        if shares_outstanding > 0 and current_price > 0:
            market_cap = shares_outstanding * current_price
    if not market_cap or market_cap == 0:
        market_cap = info_original.get('Market Cap', 0)

    # --- Cálculos de Indicadores ---
    # Dividend Yield 12 meses (já em percentual, apenas valida)
    dy_12m = info.get("trailingAnnualDividendYield", 0.0) * 100 if info.get("trailingAnnualDividendYield") is not None else 0.0
    if dy_12m > 50 or dy_12m < 0:
        dy_12m = 0.0  # Proteção contra valores absurdos

    # Dividend Yield 5 anos (evitar multiplicação por 100 e adicionar fallback)
    dy_5y = info.get("fiveYearAvgDividendYield", 0.0) if info.get("fiveYearAvgDividendYield") is not None else 0.0
    # Validação: valores fora de 0-50% são considerados inválidos
    if dy_5y > 50 or dy_5y < 0:
        print(f" (Aviso: DY 5 anos de {ticker} ({dy_5y:.2f}%) inválido, calculando manualmente...) ", end="")
        dy_5y = 0.0
        try:
            dividends = stock.dividends
            if not dividends.empty:
                # Soma os dividendos dos últimos 5 anos
                dividends_5y = dividends[dividends.index >= (datetime.now() - pd.Timedelta(days=5*365))]
                total_dividends = dividends_5y.sum()
                # Calcula preço médio dos últimos 5 anos
                avg_price = history['Close'].mean() if not history.empty else current_price
                if avg_price > 0:
                    dy_5y = (total_dividends / 5) / avg_price * 100  # Média anual de dividendos / preço médio
        except Exception as e:
            print(f" (Aviso: Falha ao calcular DY 5 anos manualmente para {ticker} - {e}) ", end="")

    growth_price = 0.0
    if not history.empty and len(history['Close']) > 1 and history['Close'].iloc[0] > 0:
        growth_price = ((history['Close'].iloc[-1] / history['Close'].iloc[0]) - 1) * 100
    payout_ratio = info.get("payoutRatio", 0.0) * 100 if info.get("payoutRatio") is not None else 0.0
    sentiment_info = get_market_sentiment_and_details(stock)

    dados = {
        "Empresa": info_original.get('Empresa', 'N/A'),
        "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'),
        "Tipo": info_original.get('Tipo', 'N/A'),
        "Market Cap": market_cap,
        "Logo": info_original.get('Logo', 'N/A'),
        "Preço Atual": current_price,
        "P/L": info.get("trailingPE"),
        "P/VP": info.get("priceToBook"),
        "DY (Taxa 12m, %)": dy_12m,
        "DY 5 Anos Média (%)": dy_5y,
        "Último Dividendo": info.get("lastDividendValue"),
        "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s', errors='coerce'),
        "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s', errors='coerce'),
        "Payout Ratio (%)": payout_ratio,
        "Crescimento Preço (%)": growth_price,
        "ROE (%)": info.get("returnOnEquity", 0.0) * 100 if info.get("returnOnEquity") is not None else 0.0,
        "Dívida Total": info.get('totalDebt'),
        "EBITDA": info.get('ebitda'),
        "Perfil da Ação": classify_stock_profile(current_price, market_cap)
    }
    dados.update(sentiment_info)
    if dados.get("EBITDA") and dados["EBITDA"] != 0 and dados.get("Dívida Total"):
        dados["Dívida/EBITDA"] = dados["Dívida Total"] / dados["EBITDA"]
    else:
        dados["Dívida/EBITDA"] = None
    return dados

def main():
    arquivo_input = r"E:\Github\Unicamp\Bussola-de-Valor\data\scanner_acoes_e_fundos_filtrado.csv"
    output_path = r"E:\Github\Unicamp\Bussola-de-Valor\data\relatorio_analise_b3.csv"
    try:
        df_input = pd.read_csv(arquivo_input)
        tickers_para_analisar = df_input['Ticker'].tolist()
    except FileNotFoundError:
        print(f"❌ ERRO CRÍTICO: Arquivo de entrada '{arquivo_input}' não encontrado.")
        return
    dados_detalhados = {}
    erros = []
    print("\nIniciando coleta de dados via yfinance...")
    for ticker in tickers_para_analisar:
        print(f"Processando: {ticker}...", end="")
        try:
            info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
            dados_detalhados[ticker] = fetch_stock_data(ticker, info_original)
            print(" ✅")
        except Exception as e:
            erros.append(ticker)
            print(f" ❌ (Erro: {e})")
    if not dados_detalhados:
        print("\n❌ NENHUM DADO FOI COLETADO COM SUCESSO. O arquivo CSV não será gerado.")
        return
    df_resultado = pd.DataFrame.from_dict(dados_detalhados, orient="index")
    df_resultado.fillna(0, inplace=True)
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_resultado.to_csv(output_path)
        print(f"\n✅ SUCESSO! Relatório salvo com sucesso em: {output_path}")
    except Exception as e:
        print(f"❌ ERRO INESPERADO AO SALVAR O ARQUIVO: {e}")

if __name__ == "__main__":
    main()


In [ ]:
    # arquivo_input = r"C:\Users\01701805\Desktop\Projetos GL\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv"
    # output_path = r"C:\Users\01701805\Desktop\Projetos GL\finance-manager\data\relatorio_analise_b3.csv"

In [ ]:
# app.py:

# df = load_data(r"C:\Users\01701805\Desktop\Projetos GL\finance-manager\data\relatorio_analise_b3.csv")


In [ ]:
# import pandas as pd

# df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\relatorio_completo_b3.xlsx')

# print(df_dados.head(2))